In [3]:
from openai import OpenAI
from apikey import api_key
from myTools import *
client = OpenAI(api_key=api_key)

In [4]:
docu =read_file("envision-brief.md")    
challenge = read_file("mychallenges/c"+"000"+".md")

# decompose challenge into question and prof_answer

question,prof_answer,references=decompose_challenge(challenge)
ref_str=create_ref(references)
print(question,prof_answer,ref_str)

define a text literal called greetings with value "Hello" and display greetings on the dashboard as a label.   ```envision
greeting = "Hello" // define the text literal
show label greeting // show the text literal as a label. There should be no 'with' !
``` Every script must display at least _one_ dashboard _tile_. The following script illustrates how to generate a dashboard that contains a single tile displaying _Hello World!_.

```envision
greeting = "Hello World!"
show label greeting
```

All the statements that start with the keyword `show` indicate that a _tile_ will be displayed. Tiles are the display mechanism provided by Envision. All the tiles present in a script are consolidated in a _dashboard_.

The value `"Hello World!"` is called a **text literal**, a value that appears verbatim in the code. The operator `=` is the **assignment operator**.


In [5]:
coder_personality="You are a proficient coder in the Domain Specific Language called Envision. \
    Your task is to generate response to the given challenge. \
    Some challenges will ask you to generate Envision code,\
    others will ask you to explain given code or answer questions related to the Envision language. \
    Do not output any intermediate thinking or explanation, only give the final answer.\
    Here is the documentation of Envision:\
    ### Documentation\n" + docu
coder_prompt=question

coder_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": coder_personality},
        {"role": "user", "content": coder_prompt}
    ],
    max_tokens=1000,  # Adjust the number of tokens based on your needs
    temperature=0.4,
)
stud_sentence=coder_response.choices[0].message.content
print(stud_sentence)


```envision
// Define a text literal called greetings with value "Hello"
greetings = "Hello"

// Display greetings on the dashboard as a label
show label "Greetings" a1b1 with greetings
```


In [8]:

# this personality sticks more to professor's answer.

judge_personality_teacherAuthority="Your goal is to judge the correctness of STUDENT ANSWER, as an answer to the QUESTION.\
In order to judge the STUDENT ANSWER, you are given the PROFESSOR ANSWER with a piece of related documentation.\
Your main job is not to check the syntax correctness, but the logical correctness.\
If the STUDENT ANSWER does not treat the QUESTION logically, it is UNACCEPTABLE.\
Pay special attention to the comments in the PROFESSOR ANSWER. If these comments include\
a rule and if the STUDENT ANSWER violates it, this is UNACCEPTABLE.\
Adding a print position label like a1b2 in the show command is ACCEPTABLE, as long as this is not forbidden by\
the QUESTION or the comments in the PROFESSOR ANSWER.\
The use of extra variable or table to temporarily contain a intermediate quantity is ACCEPTABLE.\
Differences in variable names, column names, table names and label names etc. shall systematically be ACCEPTABLE! \
There are sometimes various ways or logics to treat the same QUESTION, and this is ACCEPTABLE, as long as the goal of the QUESTION is achieved.\
Let's think aloud step by step before making your judgement. Tell each ACCEPTABLE or UNACCEPTABLE point. \
At the end of your output, you should judge 0 if there is anything UNACCEPTABLE (even only 1 mark of UNACCEPTABLE) in the STUDENT ANSWER;\
and judge 1 if everything is ACCEPTABLE.\
you must add a new line which ONLY contains either 0 or 1, strictly according to your judgment.\
This is the piece of related documentation : \n ## DOCUMENTATION\n" + ref_str

judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence

# Generate a response from the chatbot
judge_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": judge_personality_teacherAuthority},
        {"role": "user", "content": judge_prompt}
    ],
    max_tokens=800,  # Adjust the number of tokens based on your needs
    temperature=0.1,
)
print(judge_prompt)
# Print the generated response
print(judge_response.choices[0].message.content)


### QUESTION: define a text literal called greetings with value "Hello" and display greetings on the dashboard as a label.  
### PROFESSOR ANSWER: ```envision
greeting = "Hello" // define the text literal
show label greeting // show the text literal as a label. There should be no 'with' !
```
### STUDENT ANSWER: ```envision
// Define a text literal called greetings with value "Hello"
greetings = "Hello"

// Display greetings on the dashboard as a label
show label "Greetings" a1b1 with greetings
```
- The student correctly defines a text literal called `greetings` with the value "Hello".
- The student correctly displays `greetings` on the dashboard as a label.
- The student uses a print position label `a1b1` which is acceptable.
- The student uses the label "Greetings" instead of "greeting" which is acceptable as long as the content is displayed correctly.

Therefore, the student answer is **ACCEPTABLE**.

1


In [9]:
# extract the 'real' code from the student answer (cut away the '''envision bit at the start and end)
def extract_code(stud_sentence):
    lines = stud_sentence.strip().split('\n')
    return '\n'.join(lines[1:-1])
print(extract_code(stud_sentence))

// Define a text literal called greetings with value "Hello"
greetings = "Hello"

// Display greetings on the dashboard as a label
show label "Greetings" a1b1 with greetings


In [10]:
import requests
# send code to online compiler and check if it compiles

def check_compilation(script):
    url = "https://try.lokad.com/w/script/trycompile"
    payload = {
        "Script": script
    }

    try:
        # Send POST request
        response = requests.post(url, json=payload)

        # Check for successful response
        if response.status_code == 200:
            result = response.json()
            if result["IsCompOk"]:
                return True
            else:
                print("Compilation Failed!")
                for message in result["CompMessages"]:
                    print(f"Error: {message['Text']} (Line: {message['Line']}, Start: {message['Start']}, Length: {message['Length']}, Severity: {message['Severity']})")
                    return False
        else:
            print("Error: Unable to reach the compilation service.")
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

        

# Example usage
check_compilation(extract_code(stud_sentence))


Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 5, Start: 29, Length: 4, Severity: Error)


False

In [29]:
# for each question, try 3 generation-compilations.
# if compiles, further check with judge.
def pipeline_verify(challenge,coder_personality,judge_personality=judge_personality_teacherAuthority):
    question,prof_answer=decompose_challenge(challenge)
    n_tries=3

    for compile_try in range(n_tries):
        coder_prompt=question
        coder_response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": coder_personality},
                {"role": "user", "content": coder_prompt}
            ],
            max_tokens=1000,  # Adjust the number of tokens based on your needs
            temperature=0.2,
        )
        stud_sentence=coder_response.choices[0].message.content
        if (question.split("\n")[0] ==\
        '# this question expects a textual answer and not generation of code. #'):
            print('# theoretical question, no compile.')
            break
        if(check_compilation(extract_code(stud_sentence))):
            print('# compile ok')
            break
        elif (compile_try==n_tries-1):
            print( "# too many failures !")
            print('# badcode:\n'+extract_code(stud_sentence))
            return stud_sentence,"too many failures !",False

    judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence
    judge_response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": judge_personality},
            {"role": "user", "content": judge_prompt}
        ],
        max_tokens=800,  # Adjust the number of tokens based on your needs
        temperature=0.2,
    )
    judge_sentence=judge_response.choices[0].message.content
    judge_decision=judge_sentence[-1]=='1'
    print ('# judge_decision:',judge_decision)
    if not judge_decision:
        print('# badcode:\n',extract_code(stud_sentence))
        print('# judge explanation:\n',judge_sentence)
    return stud_sentence,judge_sentence,judge_decision
# a all-in-one function to score a model on a list of challenges
def pipeline_score_allchallenge(indexes,coder_personality):
    challenges=[readfile("mychallenges/c"+index+".md") for index in indexes]
    score=0
    for i in range(len(challenges)):
        challenge=challenges[i]
        print('\n### verifying challenge No. '+indexes[i])
        _,_,judge_decision=pipeline_verify(challenge,coder_personality)
        if (judge_decision): score+=1
    print('correct:'+str(score)+' out of '+str(len(challenges))+', '+str(score/len(challenges)*100)+'%')  

In [38]:
# test the pipeline. typically fails on challenges 0,4,7 
# (these questions include grammar rules not covered in the documentation)

indexes=["000","001","002","003","004","005","006","007","008"]
pipeline_score_allchallenge(indexes,coder_personality)


### verifying challenge No. 000
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 2, Start: 20, Length: 4, Severity: Error)
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 2, Start: 20, Length: 4, Severity: Error)
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 2, Start: 20, Length: 4, Severity: Error)
# too many failures !
# badcode:
greetings = "Hello"
show label "" a1b1 with greetings

### verifying challenge No. 001
# compile ok
# judge_decision: True

### verifying challenge No. 002
Compilation Failed!
Error: Cannot broadcast T data to Scalar data. (Line: 9, Start: 27, Length: 3, Severity: Error)
# compile ok
# judge_decision: True

### verifying challenge No. 003
# compile ok
# judge_decision: True

### verifying challenge No. 004
Compilation Failed!
Error: Undefined variable 'Scalar.x'. (Line: 1, Start: 10, Length: 1, Severity: Error)
Compilation Failed!
Error: Invalid token (Line: 1, Start: 12, Length: 1, Severity: Error)
Compilation 